In [4]:
path=r"/content/IMDB_Dataset[1].csv"

In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences


In [10]:
df =pd.read_csv (path, nrows=100)

In [11]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [12]:
#replace
df["sentiment"].replace({"positive":1,"negative":0},inplace=True)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [13]:
x=np.array(df["review"].values)
y=np.array(df["sentiment"].values)

In [14]:
x_filtered = []
for review in x:
  review = review.lower()
  for i in review:
    punc = '''  !()-[]{}:;'"\,<>./@#$%^&*_~  '''
    if i in punc:
      review = review.replace(i, " ")
  x_filtered.append(review)
print(x_filtered)


['one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me  br    br   the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word  br    br   it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away  br    br   i would say the main appeal of the show is due to the f

In [15]:
vocabulary_size=5000
one_hot_encoded = [one_hot(review,vocabulary_size) for review in x_filtered]
print(one_hot_encoded)

[[427, 2865, 4875, 875, 810, 2063, 3835, 402, 3013, 3570, 897, 1484, 547, 2221, 4587, 1852, 4672, 3205, 508, 1625, 467, 1566, 1429, 78, 3755, 2530, 2190, 3967, 1899, 4933, 4933, 4875, 311, 2518, 402, 329, 1899, 3765, 547, 2307, 3432, 1474, 138, 2565, 1148, 2865, 2352, 2826, 2567, 2936, 467, 1776, 4875, 4492, 219, 3485, 1899, 1429, 78, 4279, 1827, 2779, 229, 4875, 1754, 4322, 4468, 3152, 1429, 2779, 992, 610, 2543, 3967, 1312, 3579, 2150, 4268, 4468, 2352, 3432, 78, 4752, 2936, 4875, 492, 497, 2865, 4875, 4492, 4933, 4933, 1823, 78, 3662, 547, 1566, 402, 78, 4875, 1846, 4272, 3579, 4875, 2786, 4814, 2000, 2135, 3958, 1823, 3906, 1103, 2899, 2670, 390, 365, 4540, 1567, 2865, 4875, 2561, 866, 1644, 4875, 2623, 4258, 1540, 465, 138, 1493, 903, 143, 3719, 78, 4279, 1335, 2899, 4875, 3855, 4534, 390, 78, 2984, 3579, 4734, 4157, 3463, 3611, 2501, 570, 3851, 1498, 138, 1781, 143, 3326, 4158, 4942, 2466, 331, 138, 882, 4428, 1625, 4753, 4603, 1954, 4933, 4933, 3792, 2667, 2999, 4875, 1457, 3083

In [16]:
max_length = 500
x_padded = pad_sequences(one_hot_encoded, max_length, padding = "post")

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_padded, y, test_size = 0.3)

In [22]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
embedded_vector_size = 35
model = Sequential()
model.add(Embedding(vocabulary_size,embedded_vector_size,input_length= max_length))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 35)           175000    
                                                                 
 lstm (LSTM)                 (None, 100)               54400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 229,501
Trainable params: 229,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
model.compile(loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
3/3 [==============================] - 7s 170ms/step - loss: 0.6995 - accuracy: 0.3143
Epoch 2/10
3/3 [==============================] - 1s 172ms/step - loss: 0.6906 - accuracy: 0.5429
Epoch 3/10
3/3 [==============================] - 0s 167ms/step - loss: 0.6890 - accuracy: 0.5429
Epoch 4/10
3/3 [==============================] - 0s 102ms/step - loss: 0.6908 - accuracy: 0.6000
Epoch 5/10
3/3 [==============================] - 0s 166ms/step - loss: 0.6893 - accuracy: 0.5571
Epoch 6/10
3/3 [==============================] - 1s 174ms/step - loss: 0.6869 - accuracy: 0.5429
Epoch 7/10
3/3 [==============================] - 1s 164ms/step - loss: 0.6867 - accuracy: 0.6286
Epoch 8/10
3/3 [==============================] - 0s 168ms/step - loss: 0.6891 - accuracy: 0.6000
Epoch 9/10
3/3 [==============================] - 0s 163ms/step - loss: 0.6846 - accuracy: 0.6286
Epoch 10/10
3/3 [==============================] - 0s 168ms/step - loss: 0.6835 - accuracy: 0.6143


In [24]:
score =model.evaluate(x_test,y_test)
print("ACCURACY :", score)

1/1 [==============================] - 0s 441ms/step - loss: 0.6550 - accuracy: 0.7000
ACCURACY : [0.6549587845802307, 0.699999988079071]


In [27]:
model.save("IMDB_Sentiment_Analysis")

In [32]:
from keras.models import load_model
trained_model=load_model(r"/content/IMDB_Sentiment_Analysis")
predicted = trained_model.predict(x_test)[2]

sentiment = 1 if predicted > 0.6 else 0

print("PREDICTED : ",sentiment)
print("ACTUAL : ",y_test[2])


1/1 [==============================] - 0s 384ms/step
PREDICTED :  0
ACTUAL :  0


In [40]:
def get_sentiment(sentence: str):
    if isinstance(sentence, (str)):
        pass
    else:
        raise Exception("Input needs to be of type 'str' ")

    # filtering the sentence
    sentence = sentence.lower()

    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

    for word in sentence:
        if word in punc:
            sentence = sentence.replace(word, " ")

    # Loading the saved trained model.
    from keras.models import load_model

    trained_model = load_model("/content/IMDB_Sentiment_Analysis")

    predicted = trained_model.predict(x_test)[2]
    sentiment = 1 if predicted > 0.6 else 0

    if sentiment == 1:
        print("Positive")
    else:
        print("Negative")

    return sentiment


In [44]:
result =get_sentiment("Very nice movie!")
print(result)

1/1 [==============================] - 0s 332ms/step
Negative
0
